# Save the Model

To save this model so that you can use it from various locations, including other notebooks or the model server, upload it to s3-compatible storage.

## Install the required packages and define a function for the upload

In [ ]:
!pip install boto3 botocore

In [ ]:
import os
from minio import Minio
import boto3
# Obtener las variables de entorno
AWS_S3_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
# Validar que todas las variables de entorno estén configuradas
if not all([AWS_S3_ENDPOINT, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_BUCKET]):
    raise EnvironmentError("Falta una o más variables de entorno: AWS_S3_ENDPOINT, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_BUCKET")
# Asegurarse de que el endpoint no tenga una ruta o path
AWS_S3_ENDPOINT = AWS_S3_ENDPOINT.split('/')[0]  # Tomar solo la parte del dominio
# Crear el cliente de MinIO o S3
client = Minio(
    AWS_S3_ENDPOINT.replace("https://", "").replace("http://", ""),  # Asegurarse de que no haya protocolo en el endpoint
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    secure=True  # Usar HTTPS
)
# Crear el cliente de S3 con boto3
s3 = boto3.resource(
    's3',
    endpoint_url=f"https://{AWS_S3_ENDPOINT}",  # Asegúrate de que el endpoint sea solo el dominio
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
# Función para subir directorios a S3
def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    bucket = s3.Bucket(AWS_S3_BUCKET)  # Utilizando el bucket configurado en las variables de entorno
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"Uploading {file_path} -> {s3_key}")
            # Subir el archivo al bucket
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files

## Upload and check again

Use the function to upload the `models` folder in a rescursive fashion:

In [ ]:
# Validar que el directorio de modelos existe
local_models_directory = "models"
if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist. "
                     "Did you finish training the model in the previous notebook?")
# Llamar a la función de carga
num_files = upload_directory_to_s3(local_models_directory, "models")
# Validar si se subieron archivos
if num_files == 0:
    raise ValueError("No files uploaded. Did you finish training and saving the model to the 'models' directory? "
                     "Check for 'models/fraud/1/model.onnx'")

### Next Step

Now that you've saved the model to s3 storage, you can refer to the model by using the same data connection to serve the model as an API.
